In [7]:
import json
import xml.etree.ElementTree as ET
from datetime import datetime
from zoneinfo import ZoneInfo
import requests



class Train:

    def generate_image(self):
        
        api_key = 'e66ed4c930cffd68921f09b9d16496ff'
        if not api_key:
            raise RuntimeError("DB API Key not configured.")
        
        db_client_id = 'bed8ebafc8f7f59dd1b8aaf32dc2ec8f'
        if not db_client_id:
            raise RuntimeError("DB Client ID not configured.")
        
        try: 
            departures = self.get_departures(api_key, db_client_id)
            

        except Exception as e:
            raise RuntimeError("DB API request failure, please check logs.")

        # dimensions = device_config.get_resolution()
        # if device_config.get_config("orientation") == "vertical":
        #     dimensions = dimensions[::-1]

        # image = self.render_image(dimensions, "train_time.html", "train_time.css", departures)
        # if not image:
        #     raise RuntimeError("Failed to take screenshot, please check logs.")
        # return image
        return departures
    
    def get_departures(self, api_key, db_client_id):
        DB_URL='https://apis.deutschebahn.com/db-api-marketplace/apis/timetables/v1/plan/{evaNo}/{today}/{hour}'
        headers = {
            'DB-Client-Id': db_client_id,
            'DB-Api-Key': api_key,
            'accept': 'application/xml'
            }
        evaNo = '08011427'
        today = datetime.today().strftime("%y%m%d")
        hour = datetime.now(ZoneInfo("Europe/Berlin")).strftime("%H")
        
        response = requests.get(DB_URL.format(evaNo=evaNo, today=today, hour=hour), headers=headers)
        parsed_departures = self.parse_departures(response.content)
        return parsed_departures
    
    def parse_departures(self, xml_bytes):
        xml_string = xml_bytes.decode("utf-8")
        root = ET.fromstring(xml_string)
        departures = []

        for s in root.findall('s'):
            dp = s.find('dp')
            tl = s.find('tl')
            if dp is None or tl is None:
                continue

            # Zeitstempel in lesbares Format umwandeln (YYMMDDHHMM → datetime)
            pt = dp.attrib.get('pt')
            if not pt or len(pt) != 10:
                continue
            dt = datetime.strptime(pt, "%y%m%d%H%M")

            zugnummer = f"{tl.attrib.get('c', '')}{tl.attrib.get('n', '')}"
            gleis = dp.attrib.get('pp', '?')
            ppth = dp.attrib.get('ppth', '')
            ziel = ppth.split('|')[-1] if ppth else 'Unbekannt'

            departures.append({
                'zug': zugnummer,
                'abfahrt': dt.strftime('%H:%M'),
                'gleis': gleis,
                'ziel': ziel,
                'verspaetung': 0  # Kein Delay in XML enthalten
            })

            departures_sorted=sorted(departures, key=lambda x: datetime.strptime(x['abfahrt'], '%H:%M'))
        return departures_sorted
        
    
train = Train()
train.generate_image()

[{'zug': 'S30737',
  'abfahrt': '17:02',
  'gleis': '2',
  'ziel': 'Pirna',
  'verspaetung': 0},
 {'zug': 'S31753',
  'abfahrt': '17:12',
  'gleis': '2',
  'ziel': 'Schöna',
  'verspaetung': 0},
 {'zug': 'S31750',
  'abfahrt': '17:13',
  'gleis': '1',
  'ziel': 'Meißen Triebischtal',
  'verspaetung': 0},
 {'zug': 'S30739',
  'abfahrt': '17:32',
  'gleis': '2',
  'ziel': 'Pirna',
  'verspaetung': 0},
 {'zug': 'S31755',
  'abfahrt': '17:42',
  'gleis': '2',
  'ziel': 'Pirna',
  'verspaetung': 0},
 {'zug': 'S31752',
  'abfahrt': '17:43',
  'gleis': '1',
  'ziel': 'Meißen Triebischtal',
  'verspaetung': 0}]

In [ ]:
def parse_departures(self, xml_bytes):
        xml_string = xml_bytes.decode("utf-8")
        root = ET.fromstring(xml_string)
        departures = []

        for s in root.findall('s'):
            dp = s.find('dp')
            tl = s.find('tl')
            if dp is None or tl is None:
                continue

            # Zeitstempel in lesbares Format umwandeln (YYMMDDHHMM → datetime)
            timestamp = int(dp.attrib.get("pt")) * 60
            time = datetime.fromtimestamp(timestamp, tz=ZoneInfo("Europe/Berlin")).strftime("%H:%M")
            pt = dp.attrib.get('pt')
            if not pt or len(pt) != 10:
                continue
            dt = datetime.strptime(pt, "%y%m%d%H%M")

            zugnummer = f"{tl.attrib.get('c', '')}{tl.attrib.get('n', '')}"
            gleis = dp.attrib.get('pp', '?')
            ppth = dp.attrib.get('ppth', '')
            ziel = ppth.split('|')[-1] if ppth else 'Unbekannt'

            departures.append({
                'zug': zugnummer,
                'abfahrt': time,
                'gleis': gleis,
                'ziel': ziel,
                'verspaetung': 0  # Kein Delay in XML enthalten
            })
        return departures